# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [13]:
import pandas as pd
document = ET.parse( './data/mondial_database.xml' )

In [9]:
## Shows 10 countries with the lowest infant mortality rates
data = pd.DataFrame([])
for child in document.getroot():
    data = data.append(pd.DataFrame({'country_name':child.findtext('name'), 
                                     'infant_mortality':child.findtext('infant_mortality')}, 
                                    index=[0]), ignore_index=True)

data['infant_mortality'] = pd.to_numeric(data.infant_mortality)
data.sort_values('infant_mortality').head(n=10)

,country_name,infant_mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


In [11]:
## Shows 10 cities with the largest populations as of their most recent census/estimation
data = pd.DataFrame([])
for child in document.getroot():
    for grandchild in child.iterfind('city'):
        data = data.append(pd.DataFrame({'city_name':grandchild.findtext('name'), 
                                     'population':grandchild.findtext('./population[last()]')}, 
                                    index=[0]), ignore_index=True)

data['population'] = pd.to_numeric(data.population)
data.sort_values('population', ascending=False).head(n=10)

,city_name,population
176,Seoul,9708483.0
164,Al Qahirah,8471859.0
80,Bangkok,7506700.0
128,Hong Kong,7055071.0
92,Ho Chi Minh,5968384.0
212,Singapore,5076700.0
163,Al Iskandariyah,4123869.0
216,New Taipei,3939305.0
177,Busan,3403135.0
107,Pyongyang,3255288.0


In [34]:
data = pd.DataFrame([])
for child in document.getroot():
    pop = float(child.findtext('./population[last()]'))
    for ethgrp in child.iterfind('ethnicgroup'):
        perc = float(ethgrp.attrib['percentage']) / 100
        ethpop = pop * perc
        data = data.append(pd.DataFrame({'country':child.findtext('name'), 
                                     ethgrp.text:ethpop}, 
                                    index=[0]), ignore_index=True)
    
    if (child.find('name').text == 'Seychelles'):
        break

world = pd.DataFrame.sum(data, numeric_only=True)
world.sort_values(ascending=False).head(n=10)

Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.318570e+08
Japanese       1.265342e+08
Malay          1.219936e+08
dtype: float64

In [61]:
##name and country of a) longest river, b) largest lake and c) airport at highest elevation
data = pd.DataFrame([])
for child in document.iterfind('river'):
    country = child.attrib['country']
    name = child.findtext('name')
    length = child.findtext('length')
    data = data.append(pd.DataFrame({'river_name':name, 'country':country, 'length':length}, index=[0]), ignore_index=True)

data['length'] = pd.to_numeric(data.length)
print(data.sort_values('length', ascending=False).head(n=1))

lakedata = pd.DataFrame([])
for child in document.iterfind('lake'):
    country = child.attrib['country']
    name = child.findtext('name')
    area = child.findtext('area')
    lakedata = lakedata.append(pd.DataFrame({'lake_name':name, 'country':country, 'area':area}, index=[0]), ignore_index=True)

lakedata['area'] = pd.to_numeric(lakedata.area)
print(lakedata.sort_values('area', ascending=False).head(n=1))

airdata = pd.DataFrame([])
for child in document.iterfind('airport'):
    country = child.attrib['country']
    name = child.findtext('name')
    elev = child.findtext('elevation')
    airdata = airdata.append(pd.DataFrame({'airport_name':name, 'country':country, 'elevation':elev}, index=[0]), ignore_index=True)

airdata['elevation'] = pd.to_numeric(airdata.elevation)
print(airdata.sort_values('elevation', ascending=False).head(n=1))

      country  length river_name
174  CO BR PE  6448.0   Amazonas
        area         country    lake_name
54  386400.0  R AZ KAZ IR TM  Caspian Sea
    airport_name country  elevation
80  El Alto Intl     BOL     4063.0
